In [1]:
import numpy as np
import zmq
from random import choice

class FloatTensor():
    
    def __init__(self, controller, data, shape):
        
        self.controller = controller
        self.data = data
        self.shape = shape
        
        controller.socket.send_json({"functionCall":"createTensor", "data": data, "shape": shape})
        
        self.id = int(controller.socket.recv_string())
        
        print("FloatTensor.__init__: " +  str(self.id))
    
    def init_sigmoid_matrix_multiply(self, tensor_1):
        
        self.controller.socket.send_json({"functionCall":"init_sigmoid_matrix_multiply","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id]})
        return self.controller.socket.recv_string()
    
    def init_add_matrix_multiply(self, tensor_1):
        
        self.controller.socket.send_json({"functionCall":"init_add_matrix_multiply","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id]})
        return self.controller.socket.recv_string()
    
    def init_weights(self, tensor_1):
        
        self.controller.socket.send_json({"functionCall":"init_weights","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id]})
        return self.controller.socket.recv_string()
    
    def sigmoid_matrix_multiply(self, tensor_1, tensor_2):
        
        self.controller.socket.send_json({"functionCall":"sigmoid_matrix_multiply","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id, tensor_2.id]})
        return self.controller.socket.recv_string()
    
    def reset_weights(self):
        
        self.controller.socket.send_json({"functionCall":"reset_weights","objectType":"tensor","objectIndex":self.id})
        return self.controller.socket.recv_string()
    
    def inline_elementwise_subtract(self, tensor_1):
        
        self.controller.socket.send_json({"functionCall":"inline_elementwise_subtract","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id]})
        return self.controller.socket.recv_string()
    
    def multiply_derivative(self, tensor_1):
        
        self.controller.socket.send_json({"functionCall":"multiply_derivative","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id]})
        return self.controller.socket.recv_string()
    
    def add_matrix_multiply(self, tensor_1, tensor_2):
        
        self.controller.socket.send_json({"functionCall":"add_matrix_multiply","objectType":"tensor","objectIndex":self.id,"tensorIndexParams":[tensor_1.id, tensor_2.id]})
        return self.controller.socket.recv_string()
    
    def print(self):
        
        self.controller.socket.send_json({"functionCall":"print","objectType":"tensor","objectIndex":self.id})
        print(self.controller.socket.recv_string())
    
class SyftController():
    
    def __init__(self, identity):
        
        self.identity = identity
        
        context = zmq.Context()
        self.socket = context.socket(zmq.DEALER)
        self.socket.setsockopt_string(zmq.IDENTITY, identity)
        self.socket.connect("tcp://localhost:5555")
    
    def FloatTensor(self,data,shape):
        return FloatTensor(self,data,shape)


In [3]:
# connect
identity = 'worker-%d' % (choice([0,1,2,3,4,5,6,7,8,9]))
sc = SyftController(identity)

In [4]:
# create tensors

l0_tensor = sc.FloatTensor([0,0,1,0,1,1,1,0,1,1,1,1], [3,3,3,3])

l1_tensor = sc.FloatTensor([0,0,0,0], [4])

np.random.seed(1)
syn0 = 2*np.random.random((3)) - 1
syn0_tensor = sc.FloatTensor(syn0.tolist(), [3])

l1_error_tensor = sc.FloatTensor([0,0,1,1], [4])

save_error_tensor = sc.FloatTensor([0,0,1,1], [4])

FloatTensor.__init__: 0
FloatTensor.__init__: 1
FloatTensor.__init__: 2
FloatTensor.__init__: 3
FloatTensor.__init__: 4


In [5]:
# init

l0_tensor.init_sigmoid_matrix_multiply(l0_tensor)

syn0_tensor.init_add_matrix_multiply (l0_tensor)

l1_error_tensor.init_weights (save_error_tensor)

'init_weights: OK'

In [6]:
# train

for iter in range(100):
    
    l1_tensor.sigmoid_matrix_multiply (l0_tensor, syn0_tensor)

    l1_error_tensor.reset_weights ()

    l1_error_tensor.inline_elementwise_subtract (l1_tensor)

    l1_error_tensor.multiply_derivative (l1_tensor)

    syn0_tensor.add_matrix_multiply (l0_tensor, l1_error_tensor)


In [7]:
# results

print ("Output After Training:")
l1_tensor.print()
print ("Weights After Training:")
syn0_tensor.print()

Output After Training:
0.1103504, 0.09193923, 0.9256488, 0.9104123
Weights After Training:
4.621287, -0.2033847, -2.093253
